In [48]:
import numpy as np
import matplotlib.pyplot as plt 

import pandas as pd  
import seaborn as sns 

%matplotlib inline

In [49]:
from sklearn.datasets import load_boston
boston_dataset = load_boston()

In [50]:
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston['MEDV'] = boston_dataset.target
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [51]:
X = pd.DataFrame(np.c_[boston['LSTAT'], boston['RM']], columns = ['LSTAT','RM'])
Y = boston['MEDV']

In [116]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(404, 2)
(102, 2)
(404,)
(102,)


In [117]:
# import the regressor 
from sklearn.tree import DecisionTreeRegressor  
  
# create a regressor object 
regressor = DecisionTreeRegressor(random_state = 0, min_samples_leaf = 20, max_depth = 4)
  
# fit the regressor with X and Y data
regressor.fit(X_train, Y_train) 

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=20,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [118]:
from sklearn.metrics import mean_squared_error

# model evaluation for training set
y_train_predict = regressor.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

# model evaluation for testing set
y_test_predict = regressor.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

The model performance for training set
--------------------------------------
RMSE is 4.345175086273103
The model performance for testing set
--------------------------------------
RMSE is 4.342467893064587


In [119]:
df_train = pd.concat([X_train,Y_train], axis = 1)
df_train['Tree_pred'] = y_train_predict
df_train.loc[:,'Leaf_node'] = regressor.apply(X_train)

In [120]:
df_test = pd.concat([X_test,Y_test], axis = 1)
df_test['Tree_pred'] = y_test_predict
df_test.loc[:,'Leaf_node'] = regressor.apply(X_test)

In [121]:
feature_cols = ['LSTAT','RM']
target_col = 'MEDV'

In [122]:
from sklearn.neighbors import KNeighborsRegressor

df_final = pd.DataFrame()
for i in df_test['Leaf_node'].unique():
    X_train = df_train[df_train['Leaf_node']==i]
    X_test = df_test[df_test['Leaf_node']==i]
    
    if(X_train.shape[0]<5):
        X_test.loc[:,'pred'] = regressor.predict(X_test[feature_cols])
        
    else:
        neigh = KNeighborsRegressor(n_neighbors=3)
        neigh.fit(X_train[feature_cols],X_train[target_col])
        X_test.loc[:,'pred'] = neigh.predict(X_test[feature_cols])
    df_final = pd.concat([df_final,X_test], axis = 0)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [123]:
df_final.head(3)

,LSTAT,RM,MEDV,Tree_pred,Leaf_node,pred
226,3.13,8.040,37.6,45.334783,16,50.000000
273,6.58,7.691,35.2,45.334783,16,44.300000
224,4.14,8.266,44.8,45.334783,16,45.166667
262,5.91,8.398,48.8,45.334783,16,40.633333
98,3.57,7.820,43.8,45.334783,16,47.966667
163,3.32,8.375,50.0,45.334783,16,47.600000
186,4.45,7.831,50.0,45.334783,16,44.633333
292,4.70,6.630,27.9,30.476667,3,27.700000
351,5.49,6.579,24.1,30.476667,3,28.700000
157,4.59,6.943,41.3,30.476667,3,32.366667


In [106]:
(np.sqrt(mean_squared_error(df_final[target_col], df_final['Tree_pred'])))

4.342467893064588

In [107]:
(np.sqrt(mean_squared_error(df_final[target_col], df_final['pred'])))

4.49349808662096

In [108]:
# MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [109]:
mean_absolute_percentage_error(df_final[target_col], df_final['Tree_pred'])

17.30834665573485

In [110]:
mean_absolute_percentage_error(df_final[target_col], df_final['pred'])

19.037478346734652

### Random Forest

In [111]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(404, 2)
(102, 2)
(404,)
(102,)


In [112]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=3, random_state=0,
                             n_estimators=100)
regr.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [114]:
from sklearn.metrics import mean_squared_error

# model evaluation for training set
y_train_predict = regr.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

# model evaluation for testing set
y_test_predict = regr.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

print('MAPE for testing set is')
mean_absolute_percentage_error(Y_test, y_test_predict)

The model performance for training set
--------------------------------------
RMSE is 4.045059958091891
The model performance for testing set
--------------------------------------
RMSE is 3.7047247467962414
MAPE for testing set is


15.79725308476371